In [24]:

from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import pandas as pd
from googlesearch import search
import requests
from bs4 import BeautifulSoup

tqdm.pandas()


def get_espncricinfo_link(row):
    if row["Player Name"] == "Andre Siddharth":
        row["Player Name"] = "Andre Siddarth"
    if row["Team"] == "CHE":
        row["Team"] = "CSK"

    query = f"{row['Player Name']} ESPNcricinfo {row['Team']}"
    try:
        search_result = search(query, sleep_interval=2, num_results=10, lang="en", timeout=10)
        for link in search_result:
            if "espncricinfo.com" in link:
                return link
            
        print(f"Could not find ESPNcricinfo link for {row['Player Name']}.")
        return None

    except Exception as e:
        print(f"Error searching for {row['Player Name']}: {e}. Please try again.")
        print(query)
        return None
    
def get_espncricinfo_number(link):
    return int(link.split("-")[-1]) if link else None

data_dir = "../testing_data"
name = "IPL_squad"
csv_file_path = f'{data_dir}/{name}.csv'  # Replace with your input CSV file path
output_csv_path = f'{data_dir}/{name}_with_id.csv'  # Replace with your desired output CSV file path
df = pd.read_csv(csv_file_path)

# with ProcessPoolExecutor(max_workers=1) as executor:
#     df["ESPNcricinfo Link"] = list(tqdm(executor.map(get_espncricinfo_link, df["Player Name"]), total=len(df["Player Name"]), desc="Adding ESPNcricinfo Links"))
df["ESPNcricinfo Link"] = df.progress_apply(get_espncricinfo_link, axis=1)
df['key_cricinfo'] = df['ESPNcricinfo Link'].apply(get_espncricinfo_number)

print(df[["Player Name", "key_cricinfo"]])

all_players_df = pd.read_csv("../data/raw/cricsheet/people.csv")
players_df = pd.merge(df, all_players_df[["identifier", "key_cricinfo", "name"]], on="key_cricinfo", how="left")

players_df.to_csv(output_csv_path, index=False)

  1%|          | 2/228 [00:10<20:15,  5.38s/it]

Error searching for Vansh Bedi: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DVansh%2BBedi%2BESPNcricinfo%2BCSK%26num%3D12%26hl%3Den%26start%3D0%26safe%3Dactive&hl=en&q=EgRPf5NYGI6CkMAGIjB4b_KvP7Ne0INzZiOQoEJY0klXIdZBEj5ERCMlfBPipa4lwGIGEJz1U6q2Zw2oG80yAnJSWgFD. Please try again.
Vansh Bedi ESPNcricinfo CSK


  1%|          | 2/228 [00:20<38:30, 10.22s/it]


KeyboardInterrupt: 

In [ ]:

        # header = {
        #     "accept": "*/*",
        #     "accept-encoding": "gzip, deflate, br",
        #     "accept-language": "en-US,en;q=0.9",
        #     "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        #     # 'User-agent': 'your bot 0.1'
        # }
        # response = requests.get(f"https://www.google.com/search?q={query}")
        # if response.status_code == 200:

        #     soup = BeautifulSoup(response.text, 'html.parser')
        #     links = soup.find_all('a', href=lambda href: href and "espncricinfo" in href)
        
        # else:
        #     print(f"Could not find ESPNcricinfo link for {player_name}.")
        #     return None

In [7]:
import pandas as pd
from datetime import datetime

df = pd.read_csv("/home/himanshu/D11_midprep_FIFS/src/testing_data/ipl_2025_schedule_raw.csv")

df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, "%a, %b %d %Y").strftime("%Y-%m-%d"))
df["venue"] = df["Stadium"]
df["link"] = df["Match Link"]

df.drop(columns=["Stadium", "Match Link"], inplace=True)

df

interim_df = pd.read_csv("/home/himanshu/D11_midprep_FIFS/src/data/interim/ipl/all.csv")
venues = interim_df["Venue"].unique()
df["venue_new"] = df["venue"].copy()

for i in range(len(df)):
    if df["venue"][i] not in venues:
        if df["venue"][i] == "Rajiv Gandhi International Stadium, Hyderabad":
            df.loc[i, "venue_new"] = "Rajiv Gandhi International Stadium, Uppal"
        elif df["venue"][i] == "M.Chinnaswamy Stadium, Bengaluru":
            df.loc[i, "venue_new"] = "M.Chinnaswamy Stadium"
        elif df["venue"][i] == "MA Chidambaram Stadium, Chennai":
            df.loc[i, "venue_new"] = "MA Chidambaram Stadium, Chepauk"
        elif df["venue"][i] == "Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur, Chandigarh":
            df.loc[i, "venue_new"] = "Maharaja Yadavindra Singh International Cricket Stadium, Mullanpur"


for i in range(len(df)):
    if df["venue_new"][i] not in venues:
        print(f"Venue {df['venue_new'][i]} not found in interim_df.")


df.to_csv("../testing_data/ipl_2025_schedule.csv")